In [1]:
import numpy as np
import matplotlib.pyplot as plt
from hyperspy.api import load
from pixstem.api import PixelatedSTEM
%matplotlib qt

from scipy.signal.signaltools import wiener
from skimage.feature import match_template
from scipy.ndimage import gaussian_filter
from skimage.restoration import denoise_nl_means, estimate_sigma

file = load("input1.blo", lazy=True)
s = PixelatedSTEM(file.inav[30, 30])
original = np.array(s)

In [2]:
sigma_est = np.mean(estimate_sigma(original, ))
# patch_size for 580 - 1, patch_size for 144 = 3
nlm = denoise_nl_means(original, h=1.15*sigma_est, fast_mode=True, patch_size=3, patch_distance=6, )
gaussian = gaussian_filter(original, 1.15*sigma_est)
wien = wiener(original, 5, 3)

# input: gamma corrected. input2: non-gamma corrected
# spot for input and input2 - [265:320, 265:320], [177:222, 208:253], [392:435, 104:147], [225:268, 241:284]
# spot for input1 - [65:80, 65:79], [43:55, 50:63], [96:108, 25:37], [55:66, 58:70]

originalTemplate = original[65:80, 65:79]
originalResult = match_template(original, originalTemplate, pad_input=True)
ab = np.unravel_index(np.argmax(originalResult), originalResult.shape)
a, b = ab[::-1]
#######################################################################################################
nlmTemplate = nlm[65:80, 65:79]
nlmResult = match_template(nlm, nlmTemplate, pad_input=True)
cd = np.unravel_index(np.argmax(nlmResult), nlmResult.shape)
c, d = cd[::-1]
#######################################################################################################
gaussianTemplate = gaussian[65:80, 65:79]
gaussianResult = match_template(gaussian, gaussianTemplate, pad_input=True)
ef = np.unravel_index(np.argmax(gaussianResult), gaussianResult.shape)
e, f = ef[::-1]
#######################################################################################################
wienTemplate = wien[65:80, 65:79]
wienResult = match_template(wien, wienTemplate, pad_input=True)
gh = np.unravel_index(np.argmax(wienResult), wienResult.shape)
g, h = ef[::-1]



def correlation(result):
    l = []
    for i in range(len(result)):
        for j in range(len(result[i])):
            if (result[i][j] > 0.87):#change correlation value
                l.append((i, j))
    return l
#######################################################################################################
def distance(x1, y1, x2, y2):
    return np.sqrt(pow(x1 - x2, 2) + pow(y1 - y2, 2))
#######################################################################################################
def duplicates(list, result):
    i = 0
    l = []
    while i < len(list):
        j = 0
        temp = []
        point = list[i]
        while j < len(list):
            if distance(point[0], point[1], list[j][0], list[j][1]) < 15:# change minimum center distance
                temp.append(list[j])
                list.pop(j)
            else:
                j = j + 1
        max = 0
        pnt = temp[0]
        for j in range(len(temp)):
            if (result[pnt[0]][pnt[1]] < result[temp[j][0]][temp[j][1]]):
                max = result[temp[j][0]][temp[j][1]]
                pnt = temp[j]
        l.append(pnt)
    return l


originalList = correlation(originalResult)
nlmList = correlation(nlmResult)
gaussianList = correlation(gaussianResult)
wienList = correlation(wienResult)
#######################################################################################################
originalList = duplicates(originalList, originalResult)
nlmList = duplicates(nlmList, nlmResult)
gaussianList = duplicates(gaussianList, gaussianResult)
wienList = duplicates(wienList, wienResult)

In [3]:
fig = plt.figure(figsize=(18, 12))
ax1 = plt.subplot(2, 4, 1)
ax2 = plt.subplot(2, 4, 2)
ax3 = plt.subplot(2, 4, 3)
ax4 = plt.subplot(2, 4, 4)
ax5 = plt.subplot(2, 4, 5)
ax6 = plt.subplot(2, 4, 6)
ax7 = plt.subplot(2, 4, 7)
ax8 = plt.subplot(2, 4, 8)

ax1.imshow(original, cmap=plt.cm.gray, aspect='equal')
ax1.set_axis_off()
ax1.set_title('Original')
ax2.imshow(nlm, cmap=plt.cm.gray, aspect='equal')
ax2.set_axis_off()
ax2.set_title('Non-local Means')
ax3.imshow(gaussian, cmap=plt.cm.gray, aspect='equal')
ax3.set_axis_off()
ax3.set_title('Gaussian')
ax4.imshow(wien, cmap=plt.cm.gray, aspect='equal')
ax4.set_axis_off()
ax4.set_title('Wiener')

ax5.imshow(originalTemplate, cmap=plt.cm.gray, aspect='equal')
ax5.set_axis_off()
ax5.set_title('Original Template')
ax6.imshow(nlmTemplate, cmap=plt.cm.gray, aspect='equal')
ax6.set_axis_off()
ax6.set_title('Non-Local Means Template')
ax7.imshow(gaussianTemplate, cmap=plt.cm.gray, aspect='equal')
ax7.set_axis_off()
ax7.set_title('Gaussian Template')
ax8.imshow(wienTemplate, cmap=plt.cm.gray, aspect='equal')
ax8.set_axis_off()
ax8.set_title('Wiener Template')

# for i in originalList:
#     x, y = i
#     circ = plt.Circle((y, x), 0.25, edgecolor='r', facecolor='none')
#     ax1.add_patch(circ)
# for i in nlmList:
#     x, y = i
#     circ = plt.Circle((y, x), 0.25, edgecolor='r', facecolor='none')
#     ax2.add_patch(circ)
# for i in gaussianList:
#     x, y = i
#     circ = plt.Circle((y, x), 0.25, edgecolor='r', facecolor='none')
#     ax3.add_patch(circ)
# for i in wienList:
#     x, y = i
#     circ = plt.Circle((y, x), 0.25, edgecolor='r', facecolor='none')
#     ax4.add_patch(circ)

plt.show()